In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

# Approximate computing time difference and GPU memory allocation

In [2]:
def compute_pct_change(new, old):
  return np.abs((new - old) / old)

## RTE



In [3]:
# array[0] -- GPU usage (time units)
# array[1] -- Memory allocation (%)

In [4]:
rte_fnet = np.array([434, 51.8]) ## seconds
rte_bert = np.array([546, 79.81])

In [5]:
compute_pct_change(rte_fnet, rte_bert)

array([0.20512821, 0.35095853])

## QNLI

In [6]:
qnli_fnet = np.array([119, 82]) #minutes
qnli_bert = np.array([157, 87])

In [7]:
compute_pct_change(qnli_fnet, qnli_bert)

array([0.24203822, 0.05747126])

## COLA

In [8]:
qnli_fnet = np.array([443, 20]) #second
qnli_bert = np.array([550, 23])

In [9]:
compute_pct_change(qnli_fnet, qnli_bert)

array([0.19454545, 0.13043478])

# METRICS

In [10]:
train_metrics_cols = [
    'train/train_steps_per_second',
    'train/train_samples_per_second',
    'train/train_runtime',
    'train/total_flos',
]

test_metrics_cols = [
    'eval/steps_per_second',
    'eval/samples_per_second',
    'eval/runtime',
]

metrics = [
    'train/loss',
    'eval/loss',
]

In [11]:
def generate_train_test_reports(csv_filepath, task):
    df = pd.read_csv(csv_filepath, index_col=0)
    df = df.iloc[[1, 0]] # make bert the reference value
    df.index.names = ['Models']
    
    task_metric = 'eval/matthews_correlation' if task == 'cola' else 'eval/accuracy'

    curr_metrics = metrics + [task_metric]

    def add_pct_change_last_row(df):
        df.loc['pct change'] = df.pct_change().iloc[-1]
        return df
    
    list_cols = [train_metrics_cols, test_metrics_cols, curr_metrics]
    dfs_final = []

    for cols in list_cols:
        df_new = df[cols].astype(float)
        df_new = add_pct_change_last_row(df_new)
        
        dfs_final.append(df_new)

    return dfs_final

In [12]:
csv_filepaths = {
    'qnli': '../reports/qnli_wandb_export_2023-01-31T13_32_02.342+03_00.csv',
    'cola': '../reports/cola_wandb_export_2023-01-31T13_32_22.721+03_00.csv',
    'rte': '../reports/rte_wandb_export_2023-01-31T13_32_45.655+03_00.csv'
}

In [13]:
def show_reports(csv_filepath, task):
    dfs = generate_train_test_reports(csv_filepath, task)

    for df in dfs:
        display(df)

In [14]:
for task, csv_filepath in csv_filepaths.items():
    show_reports(csv_filepath, task)

,train/train_steps_per_second,train/train_samples_per_second,train/train_runtime,train/total_flos
Models,,,,
bert-base-uncased-qnli,3.548000,56.759000,9226.963100,2.406621e+16
custom-fnet-qnli,4.687000,74.982000,6984.579100,1.809201e+16
pct change,0.321026,0.321059,-0.243025,-2.482402e-01


,eval/steps_per_second,eval/samples_per_second,eval/runtime
Models,,,
bert-base-uncased-qnli,11.008000,175.8350,31.068900
custom-fnet-qnli,14.732000,235.3200,23.215200
pct change,0.338299,0.3383,-0.252783


,train/loss,eval/loss,eval/accuracy
Models,,,
bert-base-uncased-qnli,0.031900,0.556269,0.915614
custom-fnet-qnli,0.641500,0.662218,0.600952
pct change,19.109718,0.190465,-0.343663


,train/train_steps_per_second,train/train_samples_per_second,train/train_runtime,train/total_flos
Models,,,,
bert-base-uncased-cola,8.449000,135.046000,316.596500,4.554627e+14
custom-fnet-cola,11.000000,175.823000,243.171100,3.423985e+14
pct change,0.301929,0.301949,-0.231921,-2.482402e-01


,eval/steps_per_second,eval/samples_per_second,eval/runtime
Models,,,
bert-base-uncased-cola,48.269000,762.801000,1.367300
custom-fnet-cola,63.200000,998.753000,1.044300
pct change,0.309329,0.309323,-0.236232


,train/loss,eval/loss,eval/matthews_correlation
Models,,,
bert-base-uncased-cola,0.080100,0.871127,0.558035
custom-fnet-cola,0.610400,0.619749,0.000000
pct change,6.620474,-0.288567,-1.000000


,train/train_steps_per_second,train/train_samples_per_second,train/train_runtime,train/total_flos
Models,,,,
bert-base-uncased-rte,1.919000,30.637000,406.377300,1.046969e+15
custom-fnet-rte,2.526000,40.324000,308.747900,7.870694e+14
pct change,0.316311,0.316186,-0.240243,-2.482402e-01


,eval/steps_per_second,eval/samples_per_second,eval/runtime
Models,,,
bert-base-uncased-rte,6.201000,95.42300,2.902900
custom-fnet-rte,8.222000,126.52900,2.189200
pct change,0.325915,0.32598,-0.245858


,train/loss,eval/loss,eval/accuracy
Models,,,
bert-base-uncased-rte,0.521400,1.074684,0.657040
custom-fnet-rte,0.701000,0.693157,0.483755
pct change,0.344457,-0.355014,-0.263736


## Get training parameter values

- We use the same value across all tasks and models.

In [15]:
df = pd.read_csv(csv_filepaths['qnli'])

In [16]:
params = [
    'adam_beta1',
    'adam_beta2',
    'weight_decay',
    'learning_rate',
    'max_grad_norm',
    'eval_batch_size',
    'num_train_epochs',
    'train_batch_size',
    'lr_scheduler_type',
    'hidden_dropout_prob'
]

In [17]:
df[params].drop(index=1).T.rename(columns={0: 'values'})

,values
adam_beta1,0.9
adam_beta2,0.999
weight_decay,0.01
learning_rate,0.00002
max_grad_norm,1
eval_batch_size,16
num_train_epochs,5
train_batch_size,16
lr_scheduler_type,linear
hidden_dropout_prob,0.1
